In [ ]:
# Perform the DOE and generate the equations to be used


In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Calculates the response values for
# h, w, t, L, E
# Units: mm, mm, mm, mm, Pa, N
def generateResponseValues(h, w, t, L, E, F):
    # Fixed Values needed for calculation
    mt = 90.0 # Mass of Tube
    rr = 600 #mm, Distance to center line
    omega = 2 # rev/s, Rotational Speed
    g = 9.81 # m/s^2
    rho = 8.8E3 # kg/m^3
    
    # Calculations
    #Area Moment of Inertia
    wf = w+ 2*t# Flange Width
    I = wf * (math.pow(t, 3)/6.0 + t*math.pow(h+t, 2)/2.0) + t * math.pow(h, 3)/6
    # Tip Load
    #Fstatic = mt * g
    #Fdynamic = mt * (rr/1000) * math.pow(2 * math.pi * omega, 2)
    #Fcalc = Fstatic + Fdynamic # Fmax
    # Deflection
    U = 1E6 * F * math.pow(L, 3)/(3 * E * I)
    # Stress
    M = F * L # Moment
    S = M * (h/2 + t)/I # Stress
    # Beam Mass
    mb = 2E-9 * rho * L * t * (w + 2*t + h)
    # Frequency
    mbar = mt + 0.23 * mb
    a = 0.003*E*I/(mbar*math.pow(L, 3))
    freq = math.sqrt(a)/(2 * math.pi)    
    return U, S, freq


In [3]:
from pyDOE import *


In [4]:
#ccDesign_6Var = ccdesign(6, alpha='o', face='ccf')
ccDesign_6Var = lhs(6, samples=200, criterion='m')
print 'DOE Runs: %d'%(len(ccDesign_6Var))
#print ccDesign_6Var

DOE Runs: 200


In [5]:
# 6 Variables: h, w, t, L, E, F
lb = np.array([80, 40, 2, 400, 180E9, 9000])
ub = np.array([120, 60, 7, 600, 220E9, 11000])
avg = 0.5 * (lb + ub)

nDOE = len(ccDesign_6Var)

# Values in physical space
ccDesign_6Var_CodedSpace = ccDesign_6Var
ccDesign_6Var_PhysicalSpace = np.ones([nDOE, 1]) * avg + np.multiply(np.ones([nDOE, 1]) * (ub - lb) * 0.5, ccDesign_6Var)
h_PS = ccDesign_6Var_PhysicalSpace[:,0]
w_PS = ccDesign_6Var_PhysicalSpace[:,1]
t_PS = ccDesign_6Var_PhysicalSpace[:,2]
L_PS = ccDesign_6Var_PhysicalSpace[:,3]
E_PS = ccDesign_6Var_PhysicalSpace[:,4]
F_PS = ccDesign_6Var_PhysicalSpace[:,5]

deflection_PS = np.zeros([nDOE, 1])
stress_PS = np.zeros([nDOE, 1])
freq_PS = np.zeros([nDOE, 1])

#print ccDesign_6Var_PhysicalSpace
for iDOE in np.arange(nDOE):
    deflection_PS[iDOE,:], stress_PS[iDOE, :], freq_PS[iDOE, :] = generateResponseValues(h_PS[iDOE], w_PS[iDOE], t_PS[iDOE], L_PS[iDOE], E_PS[iDOE], F_PS[iDOE])
#iDOE = 1
#print ccDesign_6Var_PhysicalSpace[iDOE,:]
#a, b, c = generateResponseValues(ccDesign_6Var_PhysicalSpace[iDOE,0], ccDesign_6Var_PhysicalSpace[iDOE,1], ccDesign_6Var_PhysicalSpace[iDOE,2], ccDesign_6Var_PhysicalSpace[iDOE,3], ccDesign_6Var_PhysicalSpace[iDOE,4], ccDesign_6Var_PhysicalSpace[iDOE,5])
#print a, b, c


In [6]:
# Fit the Response Surface on this
# http://scikit-learn.org/0.17/auto_examples/model_selection/plot_underfitting_overfitting.html
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation

/software/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
from sklearn.externals import joblib
# http://scikit-learn.org/stable/modules/model_persistence.html
polynomial_features = PolynomialFeatures(degree=2, include_bias=True)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),("linear_regression", linear_regression)])

pipeline.fit(ccDesign_6Var_PhysicalSpace, freq_PS)
print polynomial_features.get_feature_names(['h', 'w', 't', 'L', 'E', 'F'])
# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline, ccDesign_6Var_PhysicalSpace, freq_PS, scoring="neg_mean_squared_error", cv=10)
print scores
print pipeline.steps[1][1].coef_
print pipeline.steps[1][1].intercept_
joblib.dump(pipeline, 'freq_rs.pkl')


pipeline.fit(ccDesign_6Var_PhysicalSpace, deflection_PS)
# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline, ccDesign_6Var_PhysicalSpace, deflection_PS, scoring="neg_mean_squared_error", cv=10)
print scores
joblib.dump(pipeline, 'deflection_rs.pkl')

pipeline.fit(ccDesign_6Var_PhysicalSpace, stress_PS)
# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline, ccDesign_6Var_PhysicalSpace, stress_PS, scoring="neg_mean_squared_error", cv=10)
print scores
joblib.dump(pipeline, 'stress_rs.pkl')


['1', 'h', 'w', 't', 'L', 'E', 'F', 'h^2', 'h w', 'h t', 'h L', 'h E', 'h F', 'w^2', 'w t', 'w L', 'w E', 'w F', 't^2', 't L', 't E', 't F', 'L^2', 'L E', 'L F', 'E^2', 'E F', 'F^2']
[-0.30966726 -0.38130969 -0.32775624 -0.17690096 -0.2451027  -0.26278895
 -0.13909684 -0.18309149 -0.44283919 -0.29681919]
[[ 2.89331326e-16 -1.27829321e-15 -1.44483418e-20  2.83091054e-22
  -1.50294541e-19  4.55671417e-11 -2.07008734e-18 -8.45375909e-18
  -2.33659933e-18  4.71527796e-21 -2.82837907e-17  3.09269406e-12
  -5.84605280e-16 -1.39688669e-18 -1.05159426e-19 -1.33273538e-17
   1.49760206e-12 -3.54062531e-16  4.71874269e-21 -1.57289608e-18
   3.05736147e-11 -1.41642599e-17 -2.07923206e-16 -8.28540220e-13
  -2.29748084e-15 -1.04876060e-22  4.22593451e-16 -4.30748626e-14]]
[27.50748961]
[-0.00137928 -0.00044107 -0.00048068 -0.0004677  -0.00055287 -0.00084743
 -0.00029349 -0.00037346 -0.00165725 -0.00039856]
[-9.35194177 -3.20673114 -2.76417588 -3.85540281 -2.8963138  -2.79723537
 -1.91371082 -3.5521

['stress_rs.pkl']

In [8]:
from sklearn.externals import joblib
# Coded Space Response Surface Fit
# http://scikit-learn.org/stable/modules/model_persistence.html
polynomial_features = PolynomialFeatures(degree=2, include_bias=True)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),("linear_regression", linear_regression)])

pipeline.fit(ccDesign_6Var_CodedSpace, freq_PS)
print polynomial_features.get_feature_names(['h', 'w', 't', 'L', 'E', 'F'])
# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline, ccDesign_6Var_CodedSpace, freq_PS, scoring="neg_mean_squared_error", cv=10)
print scores
print pipeline.steps[1][1].coef_
print pipeline.steps[1][1].intercept_
joblib.dump(pipeline, 'freq_rs_coded.pkl')


pipeline.fit(ccDesign_6Var_CodedSpace, deflection_PS)
# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline, ccDesign_6Var_CodedSpace, deflection_PS, scoring="neg_mean_squared_error", cv=10)
print scores
joblib.dump(pipeline, 'deflection_rs_coded.pkl')

pipeline.fit(ccDesign_6Var_CodedSpace, stress_PS)
# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline, ccDesign_6Var_CodedSpace, stress_PS, scoring="neg_mean_squared_error", cv=10)
print scores
joblib.dump(pipeline, 'stress_rs_coded.pkl')

['1', 'h', 'w', 't', 'L', 'E', 'F', 'h^2', 'h w', 'h t', 'h L', 'h E', 'h F', 'w^2', 'w t', 'w L', 'w E', 'w F', 't^2', 't L', 't E', 't F', 'L^2', 'L E', 'L F', 'E^2', 'E F', 'F^2']
[-0.00124547 -0.00098798 -0.00109217 -0.00116422 -0.00167871 -0.00221388
 -0.00059816 -0.00188461 -0.00277198 -0.00120081]
[[ 1.67692520e-15  1.24296317e+01  3.00789347e+00  1.71152873e+01
  -1.64572294e+01  2.75922299e+00  7.04277619e-02  3.01226427e-01
   3.95875939e-01  2.85756880e+00 -3.46811386e+00  6.31450863e-01
  -4.41378576e-03 -4.27733830e-02  7.05670855e-01 -8.93824384e-01
   1.37235059e-01 -2.71415549e-03 -1.02390412e+00 -4.47455182e+00
   7.13836006e-01 -1.25566328e-02  3.84953656e+00 -8.25870603e-01
  -1.56428119e-02 -7.44215258e-02  9.98594165e-03 -5.11969342e-02]]
[54.04478532]
[-4.89517170e-05 -1.75839784e-05 -5.07854694e-06 -4.41858567e-06
 -8.57721955e-06 -1.02943695e-05 -6.41143453e-06 -9.48992131e-06
 -4.83359807e-05 -7.03713324e-06]
[-0.21437212 -0.10764742 -0.02960895 -0.05648635 -0.

['stress_rs_coded.pkl']

In [13]:
# Offset Coefficient by 1
# First term is a constant term that is captured in intercept
coef_tmp = pipeline.steps[1][1].coef_
print coef_tmp
print coef_tmp[0,1:]

[[-8.78574196e-15 -3.13599757e+01 -1.25228149e+01 -6.58098538e+01
   2.30744047e+01  5.09394786e-02  1.12848886e+01  4.25671178e+00
   2.96186075e+00  1.09095494e+01 -4.25634350e+00  2.49601291e-01
  -1.91308236e+00  6.00319605e-01  4.43255596e+00 -1.54568194e+00
  -3.40897861e-01 -6.14021725e-01  1.95326241e+01 -7.85242973e+00
  -1.96731543e-01 -4.14727339e+00 -1.82264349e-01 -2.34825721e-02
   1.62901412e+00 -1.10479801e-01  4.89951871e-01 -8.48656607e-02]]
[-3.13599757e+01 -1.25228149e+01 -6.58098538e+01  2.30744047e+01
  5.09394786e-02  1.12848886e+01  4.25671178e+00  2.96186075e+00
  1.09095494e+01 -4.25634350e+00  2.49601291e-01 -1.91308236e+00
  6.00319605e-01  4.43255596e+00 -1.54568194e+00 -3.40897861e-01
 -6.14021725e-01  1.95326241e+01 -7.85242973e+00 -1.96731543e-01
 -4.14727339e+00 -1.82264349e-01 -2.34825721e-02  1.62901412e+00
 -1.10479801e-01  4.89951871e-01 -8.48656607e-02]


In [26]:
# Try matrix multiplication of ufloat
# http://sys-bio.org/wp-content/uploads/2017/11/errorPropagation.pdf
from uncertainties import ufloat
from uncertainties import unumpy
a = ufloat(2, 0.1)
b = ufloat(3, 0.1)
a_array = np.array([a, b])
print a_array
a_nom = unumpy.nominal_values(a_array)
a_std = unumpy.std_devs(a_array)
a_mat = unumpy.umatrix(a_nom, a_std)
print a_nom, a_std
print a_mat
print a_mat.transpose()
print a_mat.transpose() * a_mat


[2.0+/-0.1 3.0+/-0.1]
[2. 3.] [0.1 0.1]
[[2.0+/-0.1 3.0+/-0.1]]
[[2.0+/-0.1]
 [3.0+/-0.1]]
[[4.0+/-0.4 6.0+/-0.36055512754639896]
 [6.0+/-0.36055512754639896 9.0+/-0.6000000000000001]]
[[-4503599627370496.0+/-0 3002399751580330.5+/-0]
 [3002399751580330.5+/-0 -2001599834386887.0+/-0]]
